Combine:
turbulence-index
https://portfoliooptimizer.io/blog/the-turbulence-index-measuring-financial-risk/

With

HAA from
https://allocatesmartly.com/hybrid-asset-allocation/


1. At the close on the last trading day of the month, measure the momentum of US Treasury Inflation-Protected Securities (TIPS, ETF: TIP).

    The strategy uses TIPS as a “canary asset” to determine whether to allocate to either an offensive or defensive asset universe. Since 1971, the momentum of simulated TIPS would have been positive about 86% of the time, meaning the vast majority of the time we would be allocating to the offensive asset universe.
    
2. If TIPS momentum is positive, select the four assets with the highest momentum from the following offensive asset universe:
    
    US large caps (represented by SPY), US small caps (IWM), developed international stocks (EFA), emerging market stocks (EEM), US real estate (VNQ), commodities (PDBC), intermediate-term US Treasuries (IEF) and long-term US Treasuries (TLT)
    
    For each of those four assets, if momentum is positive, allocate 25% of the portfolio to the asset, otherwise allocate that portion of the portfolio to cash either intermediate-term US Treasuries (IEF) or cash, depending on which has the highest momentum (US T-Bills are used as a proxy for cash momentum) (*).
    
    This is what’s known as “dual momentum”. The asset must exhibit both positive momentum (aka “time series momentum”) as well as high momentum relative to competing assets (aka “cross-sectional momentum”).
    
    Note: Commodities and treasuries are generally considered defensive assets, so the offensive portfolio will still be diversified in many cases.

3. If TIPS momentum is negative, allocate the entire portfolio to either intermediate-term US Treasuries (IEF) or cash, depending on which has the highest momentum.

4. All positions are executed at the close. Hold all positions until the last the trading day of the following month. Rebalance monthly, even if there is no change signaled.


Stratergy is closer to this implementation:

https://indexswingtrader.blogspot.com/2023/02/introducing-hybrid-asset-allocation-haa.html

Changes:
1. Momentum is calculated from the close of the second to last day of the month
    1.1 This does not change results too much (sr drops from 1.15 to 1.1, cagr increases from 0.96 to 0.97)

##### Notes:
1. adding QQQ hurts returns
2. vol scaling doesn't add a whole lot currently
    * slightly higher sr, but lower cagr

##### ToDo

1. Position sizes
    * Given an amount of equity and current prices what are the lots we should be purchasing



2. Daily Pnl / returns series
    * It looks like some sort of volatility scailing will work for improving the sharpe ratio
    * Calculate the daily pnl and then try to predict the volatility of the retruns series
    * Inputs could be the vol of the assets, with an indicator function as to whether they are in the universe


3. Drawdown series
4. Position sizing
    * Vol based position sizing
    * min corr of all assets with positive mom
    * https://quant.stackexchange.com/questions/65680/find-k-of-n-assets-that-minimize-the-correlation-matrix


##### Done

1. What are the dates?
1.1 Currently they are the month we selected the assets in, including the returns
    Thus we are attributing the returns to the wrong month
    We should lag asset selection instead of returns

In [ ]:
from datetime import datetime

import pandas as pd
import numpy as np
import yfinance as yf
import plotly.express as px

def sr(ret, N=252):
    ret = ret[ret!=0]
    print(f'SR: {np.mean(ret) / np.std(ret) * np.sqrt(N)}')

def cagr(cpnl):
    t = ((cpnl.index[-1] - cpnl.index[0]) / 365.25)
    t = (t.seconds / (24 * 60 * 60) + t.days)
    FV = cpnl.values[-1]
    print(f'CAGR: {FV ** (1/t) - 1}')

def annual_returns(df, col='ret'):
    return df.groupby(pd.Grouper(freq='Y'), group_keys=False)[col].agg(lambda x: (1+x).prod() - 1).to_frame()

In [ ]:
# offensive_universe = ['SPY', 'IWM', 'EFA', 'EEM', 'VNQ', 'DBC', 'TLT']

US_Equities = ['SPY', 'IWM'] #, 'QQQ']
Foreign_Equities = ['VEA', 'VWO']
Alternative_Assets = ['DBC', 'VNQ']
Non_US_realestate = ['RWO']
US_Bonds = ['IEF', 'TLT']
Non_US_bonds = ['IGOV']

offensive_universe = US_Equities + Foreign_Equities + Alternative_Assets + US_Bonds + Non_US_realestate + Non_US_bonds

cash_universe = ['SHY']
# canary_asset = ['TIP']

# #### leveraged assets
# offensive_universe = ['SSO', 'UWM', 'VEA', 'VWO', 'DBC', 'URE', 'UBT', 'UST']

# cash_universe = ['BIL', 'UST']
# canary_asset = ['TIP']


whole_universe = offensive_universe + cash_universe # + canary_asset
whole_universe = list(set(whole_universe))

In [ ]:
prices_df = yf.download(tickers=whole_universe,
                        end=datetime.now())


In [ ]:
prices_df = prices_df['Adj Close']

In [ ]:
prices_df.ffill(inplace=True)

In [ ]:
px.line(prices_df.melt(ignore_index=False), y='value', color='variable', log_y=True) 

In [ ]:
#### need returns over reference period
#### covariance over a reference period

#### non-overlapping recient returns

#### use weekly returns

In [ ]:
class MOMENTUM():
    def __init__(self) -> None:
        pass

    def fit(self, X, y=None):
        return X
    
    def transform(self, X, y=None):
        self.calc_monthly_close(X)
        ret1m = self.get_return(1)
        ret3m = self.get_return(3)
        ret6m = self.get_return(6)
        ret12m = self.get_return(12)
        mom = (ret1m + ret3m + ret6m + ret12m) / 4
        return mom#.shift(1)
    
    def calc_monthly_close(self, X):
        last_month_date = X.reset_index().groupby(pd.Grouper(key = 'Date', freq='M')).Date.apply(lambda x: x[:-1])
        self.monthly_close = X.loc[(last_month_date)].resample('M').last()
        # self.monthly_close = X.resample('M').last()
    
    def get_return(self, months):
        return self.monthly_close.pct_change(periods = months)


class WEEKLY_RETURNS():
    def __init__(self) -> None:
        pass

    def fit(self, X, y=None):
        return X
    
    def calc_monthly_close(self, X):
        self.monthly_close = X.resample('W').last()

    def transform(self, X):
        self.calc_monthly_close(X)
        return self.get_return(1)#.shift(-1)

    def get_return(self, months):
        return self.monthly_close.pct_change(periods = months)
    

class PERIOD_MEAN_RETURNS():
    def __init__(self, N) -> None:
        self.N = N

    def fit(self, X, y=None):
        return X

    def transform(self, X):
        ret = X.rolling(self.N).mean()
        return ret
    

class PERIOD_RETURNS_COVARIANCE():
    def __init__(self, N) -> None:
        self.N = N

    def fit(self, X, y=None):
        return X

    def transform(self, X):
        ret_cov = X.rolling(self.N).cov()
        return ret_cov



class MONTHLY_RETURNS():
    def __init__(self) -> None:
        pass

    def fit(self, X, y=None):
        return X
    
    def calc_monthly_close(self, X):
        self.monthly_close = X.resample('M').last()

    def transform(self, X):
        self.calc_monthly_close(X)
        return self.get_return(1)#.shift(-1)

    def get_return(self, months):
        return self.monthly_close.pct_change(periods = months)
    

class MONTHLY_PICKS():
    def __init__(self, offensive_universe, cash_universe, canary_asset) -> None:
        self.offensive_universe = offensive_universe
        self.cash_universe = cash_universe
        self.canary_asset = canary_asset
        self.regime = None
        
    def get_picks(self, X):
        self.get_regime(X)
        if self.regime:
            assets = self.get_offensive_assets(X)
        else:
            assets = self.get_defensive_assets(X)
        return assets, self.regime
    
    def get_regime(self, X):
        if X[canary_asset].values[0] > 0:
            self.regime = True
        else:
            self.regime = False

    def get_defensive_assets(self, X):
        asset = X[self.cash_universe].melt()
        # asset = asset[asset.value > 0]
        asset = asset.sort_values('value').tail(1).variable.tolist()
        return asset

    def get_offensive_assets(self, X):
        assets = X[self.offensive_universe].melt()
        assets = assets[assets.value > 0]
        assets = assets.sort_values('value').tail(4).variable.tolist()
        Cash_fraction = 4 - len(assets)
        if Cash_fraction > 0:
            defensive_asset = self.get_defensive_assets(X)
            assets = assets + defensive_asset * Cash_fraction
        return assets
    

class POSITION_SIZES():
    def __init__(self) -> None:
        pass

    def get_lots(self, capital, assets, prices):
        #### assets is a dict of aggressive
        N = len(assets)
        asset_capital = capital / N
        return [int(asset_capital / prices[asset].values[0]) for asset in assets]

    def get_lots_date(self, capital, assets, prices, date):
        return self.get_lots(capital, assets, prices.loc[date])

In [ ]:
whole_universe

In [ ]:
N = 52 * 2

weekly_ret = WEEKLY_RETURNS()
weekly_ret_df = pd.concat([weekly_ret.transform(prices_df[sym]).to_frame() for sym in whole_universe], axis=1)

reference_ret = PERIOD_MEAN_RETURNS(N)
reference_ret_df = pd.concat([reference_ret.transform(weekly_ret_df[sym]).to_frame() for sym in whole_universe], axis=1)

reference_ret_cov = PERIOD_RETURNS_COVARIANCE(N)
reference_ret_cov_df = reference_ret_cov.transform(weekly_ret_df[whole_universe])

In [ ]:
def get_distance(reference_ret_df, current_ret_df, reference_ret_cov_df):
    N = current_ret_df.shape[1]
    dist = []
    Dates = current_ret_df.index.astype(str)
    for Date in Dates:
        try:
            r_minus_mean = current_ret_df.loc[Date] - reference_ret_df.loc[Date]
            dist.append([Date, (r_minus_mean @ np.linalg.inv(reference_ret_cov_df.loc[Date]) @ r_minus_mean) / N])
        except:
            print('Failed', Date)
    dist = pd.DataFrame(dist, columns=['Date', 'dist'])
    dist['Date'] = pd.to_datetime(dist.Date)
    return dist.set_index('Date')

dist = get_distance(reference_ret_df.dropna(), weekly_ret_df.dropna(), reference_ret_cov_df.dropna())

In [ ]:
px.line(dist)

In [ ]:
weekly_ret_df.shift(-1)

In [ ]:
ret_df = pd.merge(weekly_ret_df.shift(-1), left_index=True, right=dist, right_index=True, how='left')

In [ ]:
from numpy.lib.stride_tricks import sliding_window_view

data = ret_df.dist
sw = sliding_window_view(data, N, axis=0)
scores_np = (sw <= sw[..., -1:]).sum(axis=1) / sw.shape[-1]
ret_df['dist_pct_rank'] = np.nan
ret_df.iloc[(N-1):, ret_df.columns.get_loc('dist_pct_rank')] = scores_np #np.round(scores_np * 5, 0) / 5
# pd.DataFrame(scores_np, columns=['dist_pct_rank'])


In [ ]:
ret_df = ret_df.dropna()
ret_df['ret'] = ret_df.dist_pct_rank * ret_df.SHY + (1-ret_df.dist_pct_rank) * ret_df.SPY
ret_df['cpnl'] = (ret_df.ret + 1).cumprod()


In [ ]:
px.scatter(ret_df, y='dist_pct_rank')

In [ ]:
px.bar(ret_df, y='ret').show()
fig = px.line(ret_df, y='cpnl', log_y=True)#.show()
# benchmark_trace = px.line(ret_df, y='SPY').data[0]
# benchmark_trace['line']['color'] = 'black'
# benchmark_trace['line']['dash'] = 'dash'
# fig.add_trace(benchmark_trace)
fig.show()
px.bar(ret_df.ret.rolling(52).std() * np.sqrt(52)).show()
px.bar(annual_returns(ret_df, 'ret'), y='ret').show()
cagr(ret_df.cpnl.dropna())
sr(ret_df.ret.dropna(), N=52)